In [1]:
import numpy as np
import pandas as pd
import random
import os
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from joblib import dump
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
import itertools
from sklearn.utils import shuffle
from scipy import signal
%matplotlib inline


from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

from sklearn.feature_selection import SelectFdr, chi2

from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import accuracy_score

from utils.svm import preProcess, evaluate_set
from utils.visualize import showMe
from utils.augment import apply_augment
from config.default import *


%load_ext autoreload
%autoreload 2


C:\Users\Nemes\anaconda3\envs\tf\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Nemes\anaconda3\envs\tf\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\Nemes\anaconda3\envs\tf\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
C:\Users\Nemes\anaconda3\envs\tf\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
def create_labels(X):
    y = []
    for i, r in enumerate(X):
        l = np.ones(X[r].shape[0])*i
        y = y + l.tolist()
    y = np.array(y)
    return y


In [17]:
root_path = 'C:/resources/EMG/'
post_fix = '_1s_cleaned' #'_1s_new' #
classes = settings['classes']
subjects = os.listdir(root_path)
results = {} #dictionary of pandas dataframes
for subject in subjects:
    print(f'Cross validating, leaving out subject {subject}')
    pd_result = cross_validate(subject)
    results[subject] = pd_result
    

Cross validating, leaving out subject S001
54 sessions loaded for training
58 sessions loaded for validation


  0%|          | 0/58 [00:00<?, ?it/s]

Global accuracy: 98.34%
         Accuracy
Subject          
S001        83.25
S002        99.00
S004        99.00
S005        99.50
S006        99.75
S007        98.50
S008       100.00
S009        99.00
S010       100.00
S011       100.00
S101       100.00
S102       100.00
S103       100.00
S104        98.00
S105        99.75
S106        99.50
Cross validating, leaving out subject S002
57 sessions loaded for training
58 sessions loaded for validation


  0%|          | 0/58 [00:00<?, ?it/s]

Global accuracy: 99.28%
         Accuracy
Subject          
S001       100.00
S002        87.00
S004        99.25
S005        99.50
S006        99.75
S007        98.25
S008       100.00
S009        99.00
S010       100.00
S011       100.00
S101       100.00
S102       100.00
S103       100.00
S104        98.00
S105        99.50
S106        99.50
Cross validating, leaving out subject S004
54 sessions loaded for training
58 sessions loaded for validation


...

Global accuracy: 99.02%
         Accuracy
Subject          
S001       100.00
S002        99.00
S004        99.50
S005        99.50
S006       100.00
S007        98.25
S008       100.00
S009        99.00
S010       100.00
S011       100.00
S101       100.00
S102       100.00
S103       100.00
S104        91.00
S105        99.50
S106        99.25
Cross validating, leaving out subject S105
54 sessions loaded for training
58 sessions loaded for validation


  0%|          | 0/58 [00:00<?, ?it/s]

Global accuracy: 98.47%
         Accuracy
Subject          
S001       100.00
S002        99.00
S004        99.25
S005        99.25
S006       100.00
S007        98.25
S008       100.00
S009        99.00
S010       100.00
S011       100.00
S101       100.00
S102       100.00
S103       100.00
S104        97.25
S105        85.75
S106        99.25
Cross validating, leaving out subject S106
54 sessions loaded for training
58 sessions loaded for validation


  0%|          | 0/58 [00:00<?, ?it/s]

Global accuracy: 97.9%
         Accuracy
Subject          
S001       100.00
S002        99.00
S004        99.00
S005        99.50
S006       100.00
S007        98.25
S008       100.00
S009        99.00
S010       100.00
S011       100.00
S101       100.00
S102       100.00
S103       100.00
S104        97.50
S105        99.75
S106        76.75


In [54]:
final = []
for d in results:
    by_subject = results[d].groupby(['Subject']).mean()
    final.append(by_subject.loc[:,'Accuracy'].tolist())
print(np.array(final))

[[ 83.25        99.          99.          99.5         99.75
   98.5        100.          99.         100.         100.
  100.         100.         100.          98.          99.75
   99.5       ]
 [100.          87.          99.25        99.5         99.75
   98.25       100.          99.         100.         100.
  100.         100.         100.          98.          99.5
   99.5       ]
 [100.          99.          72.          99.25       100.
   98.25       100.          99.         100.         100.
  100.         100.         100.          98.          99.75
   99.5       ]
 [100.          99.          99.25        85.25        99.75
   98.5        100.          99.         100.         100.
  100.         100.         100.          98.          99.75
   99.5       ]
 [100.          99.          99.25        99.5         71.75
   98.         100.          99.         100.         100.
  100.         100.         100.          97.25        99.75
   99.25      ]
 [100.          99

In [ ]:
pd.concat([results['S001'], results['S002']])
asddd = [results['S001'].loc[:,'Accuracy'] + results['S002'].loc[:,'Accuracy']]
asddd

In [15]:
def cross_validate(subject, log = False):
    # use session4 for validation
    subject_to_val = [subject]
    train_sessions = []
    val_sessions = []

    for subject in os.listdir(root_path):
        for session in os.listdir(os.path.join(root_path,subject)):
            val_sessions.append(os.path.join(root_path,subject, session))
            if not subject in subject_to_val:
                train_sessions.append(os.path.join(root_path,subject, session))
        


    train_records = {}
    for c in classes:
        class_data = []
        for session in train_sessions:
            data = np.load(os.path.join(session,c+post_fix+'.npy'),allow_pickle=True)
            if data.shape[0] != 0:
                class_data.append(data)
            #else:
            #    print(f"No data available for train for class {c}")
        
        train_records[c] = np.concatenate(class_data)
    print(f"{len(train_sessions)} sessions loaded for training")


    val_records = {}
    for c in classes:
        class_data = []
        for session in val_sessions:
            data = np.load(os.path.join(session,c+post_fix+'.npy'),allow_pickle=True)
            if data.shape[0] != 0:
                class_data.append(data)
        if len(class_data) != 0:
            val_records[c] = np.concatenate(class_data)
        #else:
        #    print(f"No data available for validation for class {c}")

    print(f"{len(val_sessions)} sessions loaded for validation")





    n_channels = train_records["Rest"].shape[1]
    input_length = train_records["Rest"].shape[2]

    train_y = create_labels(train_records)
    train_X = np.concatenate((list(train_records.values())), axis=0)

    train_X, train_y = apply_augment(train_X, train_y)
    train_X = train_X.reshape(train_X.shape[0], n_channels*input_length)

    # param_grid = {'C': [10], 'gamma': [0.01]}

    # grid = GridSearchCV(SVC(),param_grid,refit=True,verbose=2)
    # grid.fit(train_X,train_y)
    # model = grid.best_estimator_ 
    svm = SVC(C=10, gamma=0.01)
    svm.fit(train_X, train_y)
    return evaluate_set(svm, val_sessions, classes, post_fix, log = log)
